In [119]:
# ngram
def ngram(sentence, n):
    chunk = []
    for i in xrange(0,len(sentence.decode('utf-8'))-n+1):
        chunk.append(sentence.decode('utf-8')[i:i+n])
    return chunk


# segmentWord(try cut string in two segments)
def segment(sentence):
    n=len(sentence.decode('utf-8'))
    segments=[]
    for i in xrange(0,n):
        seg1=sentence.decode('utf-8')[0:i]
        seg2=sentence.decode('utf-8')[i:n+1]
        segments.append([seg1,seg2])
    return(segments)
        
    
# word_freq
def word_freq(sentences=None,n=6):     #senteces 為包含多個sentence的list
    pieces=[]
    for i in xrange(1,n+1):
        for sentence in sentences:
            pieces.append(ngram(sentence,i))
    pieces_concat=[]
    for piece in pieces:
        for p in piece:
            pieces_concat.append(p)
    pieces_freq={}
    for pf in pieces_concat:
        pieces_freq[pf]=0
    for pf in pieces_concat:
        if pieces_freq[pf]>0:
            pieces_freq[pf]+=1
        else:
            pieces_freq[pf]=1
    return pieces_freq
        

# word_prob
def word_prob(sentences=None,n=6):
    pieces=[]
    for i in xrange(1,n+1):
        for sentence in sentences:
            pieces.append(ngram(sentence,i))
    pieces_concat=[]
    for piece in pieces:
        for p in piece:
            pieces_concat.append(p)
    pieces_freq={}
    for pf in pieces_concat:
        pieces_freq[pf]=0
    for pf in pieces_concat:
        if pieces_freq[pf]>0:
            pieces_freq[pf]+=1
        else:
            pieces_freq[pf]=1
    word_concat=""
    for sentence in sentences:
        word_concat=word_concat+senctence
    N=len(word_concat.decode('utf-8'))
    pieces_prob={}
    for pf in pieces_freq:
        pieces_prob[pf]=pieces_freq[pf]/N
    return pieces_prob

    
# cohesion  算每一個詞再斷詞後機率是否跟該詞獨立，一個字的會自動過濾掉
def word_isolation(sentences,iso=0,n=6):  #iso表示想選擇isolation value >多少的數值
    import math
    word_group=word_freq(sentences,n)
    word_concat=""
    for sentence in sentences:
        word_concat=word_concat+sentence
    N=len(word_concat.decode('utf-8'))
    word_iso={}
    for word in word_group:
        word_mi=[]
        wordseg=segment(str(word.encode('utf-8')))
        for wseg in wordseg:
            try:
                word_f = word_group[word]
                wseg1_f = word_group[wseg[0]]
                wseg2_f = word_group[wseg[1]]
                mi=math.log(N,2) + math.log(word_f,2) - math.log(wseg1_f,2) - math.log(wseg2_f,2)
                word_mi.append(mi)
            except:
                pass
        try:
            word_iso[word]=min(word_mi)
        except:
            pass
    return dict((key,value) for key, value in word_iso.iteritems() if value >iso)



# disorder
def word_disorder(sentences,n=6,iso=0,d=1):    #val為isolation value, n為ngram長度, d為亂度閥值
    #from itertools import chain
    import re
    import numpy
    import math
    pieces={}
    for i in xrange(1,n+1):
        c=[]
        for sentence in sentences:
            c=c+ngram(sentence,i)
        pieces[i]=c

    words_iso = word_isolation(sentences,iso,n)
    words_freq = word_freq(sentences,n)
    words_dis={}
    i=0
    for word in words_iso:
        i+=1
        try:
            N=len(word)
            if N<n:
                base=pieces[N+1]

                match1=[]
                match2=[]
                for b in base:

                    if bool(re.search("^"+word,b)):
                        match1.append(b)
                    if bool(re.search(word+"$",b)):
                        match2.append(b)
                freq1=[]
                for m in set(match1):
                    freq1.append(words_freq[m])

                SUM=sum(freq1)
                log_a=[-(math.log(float(a)/SUM,2)) for a in freq1]
                pre=numpy.mean(log_a)

                freq2=[]
                for m in set(match2):
                    freq2.append(words_freq[m])

                SUM2=sum(freq2)
                log_a2=[-(math.log(float(a)/SUM,2)) for a in freq2]
                post=numpy.mean(log_a2)

                if math.isnan(pre)|math.isnan(post):
                    if math.isnan(pre):
                        words_dis[word]=post
                    else:
                        words_dis[word]=pre
                else:
                    words_dis[word]=min([pre,post]) 
            else:
                pass
        except:
            pass
    return dict((key,value) for key, value in words_dis.iteritems() if value >d)


def remove_punc(sentences):
    import string 
    import re
    delEStr = '['+string.punctuation + u' '+']'
    delCStr = u'[▲►★《》（）()&%￥#@$！?,.><*{}\'`"【】　、。，：；”～‘＠＃＄％︿＆＊｀＝－＋｜／＼？『』」「\/]'
    s=[]
    for sentence in sentences:
        if type(sentence)==unicode:
            sentence=re.sub(delEStr,u'',sentence)
            sentence=re.sub(delCStr,u'',sentence)
            s.append(sentence.encode('utf-8'))
        else:
            sentence=re.sub(delEStr,u'',sentence.decode('utf-8'))
            sentence=re.sub(delCStr,u'',sentence)
            s.append(sentence.encode('utf-8'))
    return s  

In [134]:
# run
text=['''▲巴西當局於29日宣布，未來變性受刑人可以自由選擇要到男或女監獄服刑。（圖／達志示意圖）國際中心／綜合報導　今年4月巴西聖保羅變性女囚波莉納（Veronica Bolina）被揭穿多次遭男性獄卒、員警甚至是其他囚犯襲擊，不僅將她的長髮剃光、扯開她的衣服更多次遭到施暴。該起事件使得當局開始正視變性囚犯的權益，於29日推出新規定，表示這些人士可以自行選擇要在男或女監獄服刑。變性人未來不僅可以自行選擇要在男或女監服刑，也可以穿女性囚服和內衣褲，更能夠留長頭髮；但為了避免女同志在男監遭到侵犯，這些人未來仍只能在女監服刑。該措施一出爐立刻受到來自各方的肯定，當地LGBT組織表示這個決定顯示當局對男女同性戀、雙性戀與跨性別人士的尊重。►►►傳送第一手的新聞，鎖定《ET即時》粉絲團就對了！★圖片為版權照片，由達志影像供《ETtoday東森新聞雲》專用，任何網站、報刊、電視台未經達志影像許可，不得部分或全部轉載！''']
#text=["巴西當局於29日宣布，未來變性受刑人可以自由選擇要到男或女監獄服刑巴西我巴西可以巴西可巴西"]
text=remove_punc(text)

dis=word_disorder(text,n=6,iso=1,d=1)

for w in dis:
    print w, dis[w]

print "======================"
    
coh=word_isolation(text,iso=8)

for c in coh:
    print c, coh[c]

    
    

#print text

當局 1.58496250072
達志 1.08496250072
可以 1.20751874964
在男 1.08496250072
選擇要 1.08496250072
可以自 1.08496250072
男或女監 1.08496250072
未來 1.08496250072
變性 2.0
中心綜合報導 8.49984588708
使得當局開始 8.49984588708
暴該起事件 8.49984588708
組織表示這個 8.49984588708
綜合報導今年 8.49984588708
個決定顯 8.49984588708
導今年 8.49984588708
何網 8.49984588708
意圖國際中心 8.49984588708
事件 8.49984588708
個決 8.49984588708
留長頭 8.49984588708
莉納Ve 8.49984588708
件使 8.49984588708
lina被 8.49984588708
是其他 8.49984588708
意圖國際 8.49984588708
宣布 8.49984588708
粉絲 8.49984588708
別人士的尊重 8.49984588708
day 8.49984588708
年4月 8.49984588708
納Ve 8.49984588708
其他 8.49984588708
心綜合報導 8.49984588708
電視台未經 8.49984588708
夠留 8.49984588708
中心綜合 8.49984588708
傳送第一手 8.49984588708
其他囚犯襲 8.49984588708
但為了避 8.49984588708
剃光 8.49984588708
他囚犯襲 8.49984588708
lina被揭 8.49984588708
專用任何網 8.49984588708
網站 8.49984588708
國際中心 8.49984588708
專用任何 8.49984588708
尊重 8.49984588708
ronic 8.49984588708
站報刊電 8.49984588708
納Ver 8.49984588708
莉納Ver 8.49984588708
至是其 8.49984588708
將她的長髮剃 8.49984588708
他囚

In [ ]:
# disorder archive
def word_disorder(sentences,val=10,n=6):    #val為isolation value, n為ngram
    from itertools import chain
    import re
    import numpy
    
    pieces={}
    for i in xrange(1,n+1):
        c=[]
        for sentence in sentences:
            #c = chain.from_iterable([c,ngram(sentence,i)])
            c=c+ngram(sentence,i)
        pieces[i]=c
    words_iso = word_isolation(sentences,val,n)
    
    words_freq = word_freq(sentences,n)
    words_dis={}
    for i in xrange(1,n):
        words_set=dict((key,value) for key, value in words_freq.iteritems() if len(key) == i)
        for word in words_set:
            if i<n:
                base=pieces[i+1]
                match1=[]
                for b in base:
                    if bool(re.search("^"+word,b)):
                        match1.append(b)
                freq1=[]
                for m in match1:
                    freq1.append(words_freq[m])
                SUM=sum(freq1)
                log_a=[-(math.log(float(a)/SUM,2)) for a in freq1]
                pre=numpy.mean(log_a)

                match2=[]
                for b in base:
                    if bool(re.search(word+"$",b)):
                        match2.append(b)
                freq2=[]
                for m in match2:
                    freq2.append(words_freq[m])
                SUM2=sum(freq2)
                log_a2=[-(math.log(float(a)/SUM,2)) for a in freq2]
                post=numpy.mean(log_a2)

                if math.isnan(pre)|math.isnan(post):
                    if math.isnan(pre):
                        words_dis[word]=post
                    else:
                         words_dis[word]=pre
                else:
                    words_dis[word]=min([pre,post])    
            else:
                pass
    return words_dis


In [ ]:
# test disorder
import re
import numpy
import math
val=10
n=6
sentences=text
pieces={}
for i in xrange(1,n+1):
    c=[]
    for sentence in sentences:
        #c = chain.from_iterable([c,ngram(sentence,i)])
        c=c+ngram(sentence,i)
    pieces[i]=c
    #for p in pieces[i]:
    #    print p
    #print "----------"
words_iso = word_isolation(sentences,val,n)

words_freq = word_freq(sentences,n)
#for f in words_freq:
#    print f
words_dis={}
print len(words_freq)
i=0
for word in words_freq:
    i+=1
    print "no." + str(i)
    try:
        N=len(word)
        if N<n:
            base=pieces[N+1]
            print "字長度："+str(len(word))
            print "要比較的字:" , word, "^"+word, word+"$"
            print "被比較的字:"
            match1=[]
            match2=[]
            for b in base:
                print "\t", b
                print "\t","  比較:","^"+word, bool(re.search("^"+word,b))
                print "\t","  比較:","^"+word, bool(re.search(word+"$",b))
                #print "-------"
                if bool(re.search("^"+word,b)):
                    match1.append(b)
                if bool(re.search(word+"$",b)):
                    match2.append(b)
            '''print "match-pre: "
            for m in match1:
                print m
            print "match-post: "
            for m in match2:
                print m'''
                
            
            freq1=[]
            for m in set(match1):
                print "match-pre:",m,words_freq[m]
                freq1.append(words_freq[m])
            print "freq-pre:",freq1
            print "match1", match1
            
            SUM=sum(freq1)
            print "SUM-pre:",SUM
            log_a=[-(math.log(float(a)/SUM,2)) for a in freq1]
            print "log-pre:",log_a
            pre=numpy.mean(log_a)
            print "mean-pre:",pre

            freq2=[]
            for m in set(match2):
                print "match-post:",m,words_freq[m]
                freq2.append(words_freq[m])
            print "freq-post:",freq2

            SUM2=sum(freq2)
            print "SUM-post:",SUM2
            log_a2=[-(math.log(float(a)/SUM,2)) for a in freq2]
            print "log-post:",log_a2
            post=numpy.mean(log_a2)
            print "mean-post:", post
            #mean(-log2(freq1/sum(freq1)))

            if math.isnan(pre)|math.isnan(post):
                if math.isnan(pre):
                    words_dis[word]=post
                    print "亂度:", words_dis[word]
                else:
                    words_dis[word]=pre
                    print "亂度:", words_dis[word]         
            else:
                words_dis[word]=min([pre,post]) 
                print "亂度:", words_dis[word]
        else:
            print '字長度:',len(word)
            pass
        print "========"
    except:
        print "error"
        print "========"
        pass
print words_dis